In [1]:
import sys; sys.path.append("../")
import pandas as pd

from lib.match import (
    ColumnsIndex, JaroWinklerSimilarity, StringSimilarity, DateSimilarity, ThresholdMatcher
)
from lib.date import combine_date_columns
from clean.st_tammany_so_pprr import clean
from clean.st_tammany_so_cprr import clean as clean_cprr
from match.st_tammany_so import match_cprr, prepare_post_data, match_cprr_and_post

In [2]:
cprr = clean_cprr()
cprr

../clean/st_tammany_so_cprr.py:28: FutureWarning: The default value of regex will change from True to False in a future version.
  dept_df.loc[:, 'loc'] = dept_df.loc[:, 'loc'].str.replace(r'\*$', '')
../clean/st_tammany_so_cprr.py:39: FutureWarning: The default value of regex will change from True to False in a future version.
  df.loc[:, 'occur_year'] = df.raw_occur_date.fillna(
../clean/st_tammany_so_cprr.py:60: FutureWarning: The default value of regex will change from True to False in a future version.
  df.loc[:, 'charges'] = df.charges.str.replace(r'(\n|\r)\s*', ' ')


,raw_occur_date,department_code,charges,first_name,middle_name,last_name,middle_initial,department_desc,occur_year,occur_month,occur_day,agency,data_production_year,uid,complaint_uid,charge_uid
0,4/22/2011,15,Group 2 - DR.03:09.932 b - 24 hours suspension,christopher,,harmon,,Civil,2011,4,22,St. Tammany SO,2021,db6bfd6d4be2ef1732ceef26f3585769,370036a0c6db2e8c08bf3faad4eb83ef,d84739d5c09011925fc3cebefaf39d67
1,12/10/2011,20,Violation of Rules/Policy - Warning,jessica,,roden,,St. Tammany Parish Jail,2011,12,10,St. Tammany SO,2021,e172d3efd739eb25262855d2a860977f,f1f044212613f46dffcdc12b7d66df44,7799e50be93cf5b2a8a074c498cb4b73
2,12/15/2011,35,Physical Fitness Program - DR.04:02.040 - 16 h...,jason,,felts,,Criminal Patrol,2011,12,15,St. Tammany SO,2021,805b870f51997cc3ede1ea3a94b6af27,248625e1159db998b302eb3677f39563,beb8d365a0fce45bc0d46f0f0c78981e
3,12/20/2011,40,Physical Fitness Program - DR.04:02.040 - 16 h...,rachel,,smith,,Detectives,2011,12,20,St. Tammany SO,2021,d0c150ea7cd3f0237edcf4b3a94bfe4f,574e5a78b15884e80406d025939a0223,757328c483afec1e398905df554e582d
4,1/23/2012,20,Group 2- DR.03:09.032 a. - Insubordination - f...,billie,,anglin,,St. Tammany Parish Jail,2012,1,23,St. Tammany SO,2021,02c998d88b43a4939e4f3e2fbf963680,9fb0acc28e273adece48339833443599,c0d699da6427c575651276c1c71d9430
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1818,2/22/2021,NaN,DR.03:09.916.01 B - Inadequate or unsatifactor...,shannon,,steadman,,,2021,2,22,St. Tammany SO,2021,6ae3fc00af46b26b62c50e9167f9c64e,1a5bcdd039f9412c1771ecf16b244411,3903c1145b378614def4bf3d55ff6a16
1819,2/4/2021,NaN,Group 2- DR.03:09.032 a. - Insubordination - f...,stacey,,griffin,,,2021,2,4,St. Tammany SO,2021,f917aafa619588d84993d64a743e8164,fc9fcc283b708affaf706478c5a0ca51,332a3b56a5e8ad7d9f9846b10912f14c
1820,5/31/2020,NaN,DR.03:09.933.01 insulting language or conduct ...,taylor,,poche,,,2020,5,31,St. Tammany SO,2021,a60381f1718a08b9c232607559f31ebf,9738177c01fc08529da4e19bd4bee8bc,05db2485580bff20cc1f052333b28c71
1821,1/28/2021,NaN,DR.03:09.931.01 Group 2 Fialure to Act © Fail...,thomas,,williams,,,2021,1,28,St. Tammany SO,2021,a99564b4d94fd5ed653f220c14313a27,849b9b6487cbc4dcd019c29773998b35,6d8128a11d6d3b37bd2049985093f466


In [3]:
cprr[cprr.charges.fillna('').str.contains('\r')]

,raw_occur_date,department_code,charges,first_name,middle_name,last_name,middle_initial,department_desc,occur_year,occur_month,occur_day,agency,data_production_year,uid,complaint_uid,charge_uid


In [6]:
cprr.iloc[60:80]

,raw_occur_date,department_code,charges,first_name,middle_name,last_name,middle_initial,department_desc,occur_year,occur_month,occur_day,agency,data_production_year,uid,complaint_uid,charge_uid
60,8/10/2012,37,DR.03:09.973 B - payroll fraud - demoted,jack,,admire,,Crossing Guards,2012,8,10,St. Tammany SO,2021,eb4e8ed3adbf5afec73d349e13349f01,daca9a3a49bd34b1f1a1f5550eaf4636,ef491f591a4c11520139ce081a50f771
61,8/14/2012,28,"Disruptive work behavior, violation of rules/p...",dominique,,waguespack,,Radio Room/911,2012,8,14,St. Tammany SO,2021,66a3c00db909d628389368f406d3bbeb,530f65f85190afa42748f4b878c62e1a,e5c9290f0bc4fd36dbb76d91505fbebc
62,8/1/2012,47,DR.03:09.931.01 Group 2 Fialure to Act © Fail...,brian,,o cull,,Detectives,2012,8,1,St. Tammany SO,2021,5e8945ad9fc7bec324a46124f12de50e,e5ea7fe0ad3cb1d17d26b3646bb9077e,d68d078e2e291c3568b76ae9f1e93460
63,8/15/2012,09,"Using rude, discourteous, racial related or in...",peggy,,price,,Criminal Records,2012,8,15,St. Tammany SO,2021,306c48f74176fabf162c35ce8ceaac27,4507ca19e6931023d95bf2b12cbba431,a91b69161d9addf767cd73b8a57d1c26
64,8/21/2012,09,cash balances procedures - SOP.0085.01.010.01 ...,hope,,touzet,,Criminal Records,2012,8,21,St. Tammany SO,2021,507bb284034cd34552d938a9d7dbdbb6,409ad6530e35eb9848e4491287f283ae,4f8b1a4450600ca202caf040dd6853b9
65,8/30/2012,15,individual deputy to whom the weapon is assign...,brett,,bahm,,Civil,2012,8,30,St. Tammany SO,2021,c7f343328c3cb2b54f892e98c5eba48e,10665c9bdcc9ede4e05439b47e01dc6b,75f089fa58efa1b9cd21c23909604264
66,9/19/2012,35,"Inadequate or unsatisfactory work performance,...",scott,,daussin,,Criminal Patrol,2012,9,19,St. Tammany SO,2021,b31ef01135d7cd05fe3c0de56959355d,fd1abe61571a89c109b0cc366b6f6c1b,e10a6b1c31105f5c28b5785a87b1cdff
67,9/21/2012,42,DR:03.09.955.01 - Immoral conduct/taking any a...,christa,,lee,,Crime Lab,2012,9,21,St. Tammany SO,2021,4c370fdd613ee6373ca78b68941d3014,56e2f88d85ea8993edb2777d82d00406,8f5497f5883fb78574bc6190764f7d1c
68,9/21/2012,42,Group 2 - DR.03:09.931.01 B - Failure to Act w...,diane,,futch,,Crime Lab,2012,9,21,St. Tammany SO,2021,0dd0c149811590807bb89aeeff2e2469,e4e28d73f5ca9e82bbb84f0c2d0e34d7,e1fd1f849fe7d23cdde6f6d413bda333
69,9/24/2012,24,DR.03.09.957.01 - Willful disregard of Agency ...,john,,bowers,,Work Release,2012,9,24,St. Tammany SO,2021,0b186abad2e36404a29cb6a629f73abf,dbfdbf254d3c8e95bb0a491f21716dde,072ffd9ed9a8e6d2bd40aede45aa8a10
